# Locomotion

Inspiration: 

* [Dataset](https://bitbucket.org/jonathan-schwarz/edinburgh_locomotion_mocap_dataset/overview)

In [ ]:
from os.path import join
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## Play

### 1. Example from repo

Extract two sequences from
clips and render in matplotlib.pyplot

In [ ]:
from mocap.AnimationPlotLines import animation_plot

train_data = np.load('mocap/data/edinburgh_locomotion_train.npz')
test_data = np.load('mocap/data/edinburgh_locomotion_test.npz')

clips = train_data['clips']
clips = np.swapaxes(clips, 1, 2)

index = 0
seq1 = clips[index:index+1]
seq2 = clips[index+1:index+2]

animation_plot([seq1, seq2], interval=15.15)

### 2. Dataset

There are two items in the dataset `X`- a vector of
joint coordinates and `C` - a vector of control signals.

In [ ]:
class MocapDataset(Dataset):
    def __init__(self, root='mocap/data', train=True):
        if train:
            dpath = join(root, 'edinburgh_locomotion_train.npz')
        else:
            dpath = join(root, 'edinburgh_locomotion_test.npz')
        self.data = torch.from_numpy(np.load(dpath)['clips'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        joint_positions = self.data[idx][:, :-3]
        ctrl = self.data[idx][:, -3:]
        return {'j_pos': joint_positions, 'ctrl':ctrl}

import random
mocap_dataset = MocapDataset()

idx = random.randint(0, len(mocap_dataset))
d = mocap_dataset[idx]
j = d['j_pos']
c = d['ctrl']

print('Joint shape: ', j.shape)
print('Joint mean: ', j.mean())
print('Joint min: {}, max: {} '.format(j.min(), j.max()))
print('Control shape: ', c.shape)
print('Contorl mean: ', c.mean())
print('Contorl min: {}, max: {} '.format(c.min(), c.max()))

### 2. DataLoader

Decide batch size and parallel workers.

collate\_fn

Import for efficient training. Keras is easier at these
steps.

In [ ]:
class MocapDataLoader(DataLoader):
    def __init__(self, dset, batch_size, seq_len=1, overlap_len=0, *args, **kwargs):
        super().__init__(dset, batch_size, *args, **kwargs)
        self.seq_len = seq_len
        self.overlap_len = overlap_len

    def __iter__(self):
        for batch in super().__iter__():
            jpos = batch['j_pos']
            ctrl = batch['ctrl']
            batch_size, n_frames, n_joints = jpos.shape

            reset = True
            for i in range(self.overlap_len, n_frames, self.seq_len):
                j = jpos[:, i, :]
                c = ctrl[:, i, :]
                yield j, c, reset
                reset = False

batch_size = 32
dataloader = MocapDataset(mocap_dataset, batch_size, seq_len=1)

### 3. Model

Learning:

* Dilated convolutions
* 1D convolutions